# Classifying text lines

A classifier is trained to classify single text lines of a report. For example, the lines can be classified as containing a diagnosis (`"dm"`) or history (`his`) of a patient. This task was used as a preprocessing step to later steps of structured information extraction. So that only lines classified as containing a diagnosis will be fed to a downstream classifier, extracting the exact diagnosis. This step might be unnecessary with modern transformers that can handle longer text inputs.

The files containing the necessary information are inside the `data/raw/labelling` directory. It contains manually labelled reports from different sessions.

The original classes per label are:

| category    | subcategory       | abbreviation |
|-------------|-------------------|--------------|
| diagnosis   | MS diagnosis      | dm           |
|             | other             | do           |
| current state     |              | cu           |
| history     |                   | his          |
| symptoms    | MS related        | sym          |
|             | other             | so           |
| MRI | results                  | mr           |
| lab | results                  | labr         |
|             | other             | labo         |
| medication  | MS related        | medms        |
|             | other             | medo         |
| test, treatment        | results | tr           |
| header      |                   | head         |
| unknown     |                   | unk          |

I will group the classes according to the original approach, and drop observations with no text or label.

In [ ]:
import sys
import os
sys.path.append(os.getcwd()+"/../..")
from src import paths
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import json

In [ ]:
def load_line_labelling():
    """Loading the data from the nested csv files in the different "imported_time" directories. Labelled reports have a "rev.csv" ending.
    """
    df_list = []

    for root, dirs, files in os.walk(paths.DATA_PATH_LABELLED):
        for file in files:
            # Get the research id from filename
            rid = file.split("_")[0]
            
            if (file.endswith("rev.csv") and "mri" not in file):
                # Create a dataframe from the csv file
                _df = pd.read_csv(os.path.join(root, file))
                
                # Add the rid to the dataframe
                _df = _df.rename(columns={"text": "text", "class": "class"})
                _df['rid'] = rid
                
                # Append the dataframe to the main dataframe
                try: 
                    df_list.append(_df)
                except:
                    print("Error with file: ", file)
                    print("df head: ", _df.head(5))
                    print("_df head: ", _df.head(5))
                    continue
    return pd.concat(df_list)[["text", "class", "rid"]]

In [ ]:
def clean_line_text(df: pd.DataFrame):
    """Cleans the dataframe from the load_line_labelling function. 
    Text is cleaned by:
    1) Removes whitespace from beginning and end of text
    2) removes double spaces,
    3) remove empty lines and lines starting with "·" or "··".

    Args:
        df (pd.DataFrame): Input dataframe
    """

    df = (
        df
        .fillna("")
        .assign(text=lambda d: d["text"].str.strip()
                                        .str.replace("  ", " ")
                                        .str.replace("·", "")
                                        .str.replace("··", ""))
        

    )

    return df

def clean_line_class(df: pd.DataFrame):
    """Cleans the dataframe labels in "class".
    1) Removes whitespace from beginning and end of text
    2) Correct spelling mistakes
    3) Exclude classes that are not part of the original approach
    4) Create a new column "class_agg" with the aggregated classes of the original approach.
    5) OneHotEncode the "class_agg" column

    Args:
        df (pd.DataFrame): Input dataframe
    """

    # Class mapping spelling mistakes
    class_mapping_spelling = {
        'memds': 'medms',
    }

    # Classes of original approach abbreviation
    classes_orig = ["dm", "do", "cu", "his", "sym", "so", "mr", "labr", "labo", "medms", "medo", "tr", "head", "unk"]

    # Class mapping of original approach
    class_mapping_agg = {
        'his': 'his_sym_cu',
        'sym': 'his_sym_cu',
        'cu': 'his_sym_cu',
        'labr': 'labr_labo',
        'labo': 'labr_labo',
        'to': 'to_tr',
        'tr': 'to_tr',
        'medo': 'medo_unk_do_so',
        'unk': 'medo_unk_do_so',
        'do': 'medo_unk_do_so',
        'so': 'medo_unk_do_so',
    }

    # Cleaning the class column
    df['class'] = df['class'].str.strip() \
                             .replace(class_mapping_spelling) \
                             .apply(lambda s: s if s in classes_orig else "")

    # Creating a new column with the aggregated classes
    df['class_agg'] = df['class'].replace(class_mapping_agg)

    return df

def load_clean_line_df():
    """Loads and cleans the dataframe from the load_line_labelling function.
    """
    df = load_line_labelling()
    df = clean_line_text(df)
    df = clean_line_class(df)

    return df[["rid", "text", "class", "class_agg"]]

In [ ]:
# Output
df = load_clean_line_df()
display(df.head(5))

# As there are empty labels for some lines, we will remove them for training and performance. But because old approach used them for visualization,
# we will keep them in a separate file.
# Save the dataframe
df.to_csv(os.path.join(paths.DATA_PATH_PREPROCESSED, "line_labelling_clean_withNone.csv"))

# Remove the empty labels
df = df[df["class_agg"] != ""]
df.to_csv(os.path.join(paths.DATA_PATH_PREPROCESSED, "line_labelling_clean.csv"))

In [ ]:
# Create HuggingFace Dataset
dataset = Dataset.from_pandas(df).remove_columns(["__index_level_0__"])

# Split the dataset into train, val and test
dataset = dataset.train_test_split(test_size=0.2, seed=42, shuffle=True)
train_val_dataset = dataset["train"].train_test_split(test_size=0.2, seed=42, shuffle=True)
dataset = DatasetDict({"train": train_val_dataset["train"], "validation": train_val_dataset["test"], "test": dataset["test"]})

# Encode the labels
enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
enc.fit(np.stack(dataset["train"]["class_agg"]).reshape(-1, 1))
dataset = dataset.map(lambda e: {"labels": enc.transform(np.stack(e["class_agg"]).reshape(-1, 1)).astype(int)}, batched=True)

# Save the dataset
dataset.save_to_disk(os.path.join(paths.DATA_PATH_PREPROCESSED, "line_labelling_clean_dataset"))

# Save the class mapping in a json file
class_mapping = {i: c for i, c in enumerate(enc.categories_[0])}
with open(os.path.join(paths.DATA_PATH_PREPROCESSED, "line_labelling_clean_class_mapping.json"), 'w') as fp:
    json.dump(class_mapping, fp)